In [1]:

## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


tss_annotation = doc['tss_annotation']

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [2]:
tissues

['BMDM1hKLA',
 'BMDMwt',
 'Brain',
 'FemaleReproductive',
 'Heart',
 'Kidney',
 'Liver',
 'Lung',
 'MiscOrgans',
 'Muscle',
 'Pancreas',
 'Skin',
 'Spleen',
 'Total',
 'CHO']

In [3]:
merged_expr = pd.read_csv("Results/merged/samples.merge.peaksexpression",sep="\t",index_col=0)
merged_expr.head()

,f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1,f04_peaks/Liver_START2_and_Liver_startinput1,f04_peaks/Brain_GROCap1_and_Brain_GRO1,f04_peaks/Liver_START1_and_Liver_startinput1,f04_peaks/Lung_GROCap1_and_Lung_GRO1,f04_peaks/Brain_START3_and_Brain_startinput1,f04_peaks/Heart_START1_and_Heart_startinput1,f04_peaks/CHO_GROCap2_and_CHO_GRO1,f04_peaks/Heart_START2_and_Heart_startinput1,f04_peaks/Liver_GROCap1_and_Liver_GRO1,...,f04_peaks/MiscOrgans_START2_and_MiscOrgans_startinput1,f04_peaks/Spleen_START1_and_Spleen_startinput1,f04_peaks/Lung_START2_and_Lung_startinput1,f04_peaks/CHO_START1_and_CHO_startinput1,f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_startinput1,f04_peaks/Kidney_START2_and_Kidney_startinput1,f04_peaks/Kidney_GROCap1_and_Kidney_GRO1,f04_peaks/CHO_GROCap1_and_CHO_GRO1,f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1,f04_peaks/Spleen_START2_and_Spleen_startinput1
0,1.41,0.0,1.270,0.0,6.52,0.00,1.67,31.8,0.0,0.00,...,0.00,0.00,0.0,5.38,0.00,0.0,6.39,33.00,0.00,0.00
1,1.97,0.0,2.060,3.3,0.00,0.00,0.00,0.0,0.0,0.00,...,5.98,1.04,40.5,0.00,5.65,0.0,0.00,0.00,0.00,5.71
2,1.41,0.0,0.952,0.0,3.78,0.00,0.00,22.6,0.0,0.00,...,0.00,0.00,0.0,6.83,1.72,0.0,1.83,21.60,5.64,2.51
3,3.95,0.0,0.000,0.0,0.00,3.24,1.67,0.0,0.0,0.00,...,2.10,3.45,0.0,3.14,0.00,0.0,0.00,2.63,2.44,2.29
4,165.30,0.0,1.860,81.8,0.00,43.50,51.20,0.0,0.0,7.27,...,58.80,18.20,0.0,91.90,0.00,0.0,1.52,4.78,3.38,158.90


In [4]:
tissue_map = dict()
tissue_map_short = dict() #Remove the f04_peals
for i in merged_expr.columns.values:
    for t in tissues:
        if t in i:
            if i in tissue_map:
                print("Another tissue name was found.", i)
                break
            tissue_map[i] = t
            tissue_map_short[os.path.basename(i)] = t

In [5]:
tissue_map

{'f04_peaks/BMDM1hKLA_GROCap1_and_BMDM1hKLA_GRO1': 'BMDM1hKLA',
 'f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1': 'BMDMwt',
 'f04_peaks/Brain_GROCap1_and_Brain_GRO1': 'Brain',
 'f04_peaks/Brain_START1_and_Brain_startinput1': 'Brain',
 'f04_peaks/Brain_START1_and_Brain_startinput2': 'Brain',
 'f04_peaks/Brain_START2_and_Brain_startinput1': 'Brain',
 'f04_peaks/Brain_START2_and_Brain_startinput2': 'Brain',
 'f04_peaks/Brain_START3_and_Brain_startinput1': 'Brain',
 'f04_peaks/Brain_START3_and_Brain_startinput2': 'Brain',
 'f04_peaks/CHO_GROCap1_and_CHO_GRO1': 'CHO',
 'f04_peaks/CHO_GROCap1_and_CHO_GRO2': 'CHO',
 'f04_peaks/CHO_GROCap2_and_CHO_GRO1': 'CHO',
 'f04_peaks/CHO_GROCap2_and_CHO_GRO2': 'CHO',
 'f04_peaks/CHO_START1_and_CHO_startinput1': 'CHO',
 'f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_startinput1': 'FemaleReproductive',
 'f04_peaks/FemaleReproductive_START2_and_FemaleReproductive_startinput1': 'FemaleReproductive',
 'f04_peaks/Heart_START1_and_Heart_startinput1': 'H

In [6]:
tissue_map_short

{'BMDM1hKLA_GROCap1_and_BMDM1hKLA_GRO1': 'BMDM1hKLA',
 'BMDMwt_GROCap1_and_BMDMwt_GRO1': 'BMDMwt',
 'Brain_GROCap1_and_Brain_GRO1': 'Brain',
 'Brain_START1_and_Brain_startinput1': 'Brain',
 'Brain_START1_and_Brain_startinput2': 'Brain',
 'Brain_START2_and_Brain_startinput1': 'Brain',
 'Brain_START2_and_Brain_startinput2': 'Brain',
 'Brain_START3_and_Brain_startinput1': 'Brain',
 'Brain_START3_and_Brain_startinput2': 'Brain',
 'CHO_GROCap1_and_CHO_GRO1': 'CHO',
 'CHO_GROCap1_and_CHO_GRO2': 'CHO',
 'CHO_GROCap2_and_CHO_GRO1': 'CHO',
 'CHO_GROCap2_and_CHO_GRO2': 'CHO',
 'CHO_START1_and_CHO_startinput1': 'CHO',
 'FemaleReproductive_START1_and_FemaleReproductive_startinput1': 'FemaleReproductive',
 'FemaleReproductive_START2_and_FemaleReproductive_startinput1': 'FemaleReproductive',
 'Heart_START1_and_Heart_startinput1': 'Heart',
 'Heart_START2_and_Heart_startinput1': 'Heart',
 'Kidney_GROCap1_and_Kidney_GRO1': 'Kidney',
 'Kidney_START1_and_Kidney_startinput1': 'Kidney',
 'Kidney_START2_and

In [7]:
pickle.dump([tissue_map,tissue_map_short],open("Results/tissue_map.p","wb"))